# Performance Analysis Notebook

This notebook loads and analyzes the training logs from the gesture recognition experiments.

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load the training log
log_file = Path("logs/training_log.json")
if log_file.exists():
    with open(log_file, 'r') as f:
        log_data = json.load(f)
    print(f"Loaded {len(log_data)} log entries")
else:
    print("Log file not found")
    log_data = []

In [ ]:
# Convert to DataFrame for analysis
if log_data:
    # Flatten the nested structure
    records = []
    for entry in log_data:
        base = {
            'timestamp': entry['timestamp'],
            'mode': entry['mode'],
            'parallel': entry.get('parallel', False),
            'num_models': len(entry['models'])
        }
        for model in entry['models']:
            if model in entry:
                model_data = entry[model]
                record = base.copy()
                record['model'] = model
                record['hyperparameters'] = model_data.get('hyperparameters', {})
                if 'cv_metrics' in model_data:
                    record['cv_mean_val_acc'] = model_data['cv_metrics']['mean_val_acc']
                    record['cv_std_val_acc'] = model_data['cv_metrics']['std_val_acc']
                if 'pico_accuracy' in model_data:
                    record['pico_accuracy'] = model_data['pico_accuracy']
                records.append(record)
    
    df = pd.DataFrame(records)
    print("Log data loaded into DataFrame:")
    print(df.head())
else:
    df = pd.DataFrame()
    print("No log data to process")

In [ ]:
# Basic analysis
if not df.empty:
    print("Summary of experiments:")
    print(f"Total runs: {len(log_data)}")
    print(f"Models tested: {df['model'].unique()}")
    print(f"Modes used: {df['mode'].unique()}")
    
    # Group by model and show average performance
    if 'cv_mean_val_acc' in df.columns:
        print("\nAverage CV Validation Accuracy by Model:")
        print(df.groupby('model')['cv_mean_val_acc'].mean())
    
    if 'pico_accuracy' in df.columns:
        print("\nAverage Pico Accuracy by Model:")
        print(df.groupby('model')['pico_accuracy'].mean())
else:
    print("No data to analyze")